In [35]:
import polars as pl
import seaborn as sns

In [36]:
df = pl.read_csv("/home/surayuth/her2/results/mth_feat|level_3/3_0_color.csv")

In [37]:
df \
    .group_by("case") \
    .agg(
        pl.col("label").first(),
        pl.col("ihc_score").first(),
        pl.col("prob").mean().alias("avg")
    ) \
    .with_columns(
        pl.when(pl.col("avg") > 0.5).then(1).otherwise(0)
        .alias("pred")
    ) \
    .sort("label", "ihc_score") \
    .to_pandas()

,case,label,ihc_score,avg,pred
0,3 Sep HER2 0 Case 1,0,0,0.492904,0
1,21 Sep HER2 score 0 case 3,0,0,0.502632,1
2,11 Oct HER2 0 case 3,0,0,0.488291,0
3,21 Sep HER2 score 0 case 2,0,0,0.492553,0
4,26 Sep HER2 1+ case 5,0,1+,0.495154,0
5,26 Sep HER2 1+ case 3,0,1+,0.517590,1
6,26 Sep HER2 1+ case 1,0,1+,0.514197,1
7,11 Oct HER2 1+ case 1,0,1+,0.489916,0
8,17 Jan 23 HER2 2+ Dish - with Control,0,2-,0.521569,1
9,HER2 2+ DISH- case 02,0,2-,0.521192,1


In [38]:
# from glob import glob

# paths = glob("../Data_Chula/*/*")
# df = pl.DataFrame({"path": paths})
# df.write_csv("selected_data_path.csv")